In [1]:
# default_exp utils

This contains general utilities used in different modules

# Import

In [2]:
# export
import hashlib
import json
import math
import os
import re
import time
import warnings
from pathlib import Path

import ipykernel
import nbdev.export
import numpy as np
import requests
import seaborn as sns
import torch
from IPython.display import Javascript, display
from notebook.notebookapp import list_running_servers
from PIL import Image
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from torchvision import transforms

In [3]:
from IPython.core.debugger import set_trace

# numpy/torch conversion stuff

In general I'd like to have functions which work for both numpy and torch since the APIs aren't exactly the same. The approach I've taken is to write the function in `torch` (if possible) then add a function decorator `numpyify` to allow it to work with `numpy` arrays. This approach is far from perfect but it seems to work for most my cases.

`args_loop` assumes arguments are grouped in tuples or dictionaries, which is mostly true as `*args` and `**kwargs` are tuples and dictionaries, respectively. This will loop over and apply `callback` to each argument. Again, not perfect, but seems to work for most cases.

In [4]:
# export
def args_loop(args, callback):
    if isinstance(args, tuple):  return tuple(args_loop(arg, callback) for arg in args)
    elif isinstance(args, dict): return {key: args_loop(arg, callback) for key,arg in args.items()}
    else:                        return callback(args)

`Formatter` is a base class which will format input arguments; it also keeps track if any arguments were successfully formatted. Again, not perfect but works in most cases.

In [5]:
# export
class Formatter():
    def __init__(self): self.formatted = False
    
    def predicate(self, arg): raise NotImplementedError('Please implement predicate')
    def formatter(self, arg): raise NotImplementedError('Please implement formatter')
        
    def callback(self, arg):
        if self.predicate(arg): 
            self.formatted = True
            return self.formatter(arg)
        else:                   
            return arg
        
    def __call__(self, args):
        self.formatted = False
        return args_loop(args, self.callback)

`torch2np` will convert arguments from torch to numpy arrays. It should work for all cases (tensors which require gradients and cuda tensors) and will use the same underlying data and `dtype`.

NOTE: The `formatted` flag for `torch2np` will not be thread safe, calling `torch2np = Torch2np()` per invocation should make it safe

In [6]:
# export
class Torch2np(Formatter):
    def predicate(self, arg): return isinstance(arg, torch.Tensor)
    def formatter(self, arg): return arg.detach().cpu().numpy()
torch2np = Torch2np()

`np2torch` will convert to tensor with the same `dtype`. It's not as general as `torch2np` since output tensor might need to be on the gpu.

NOTE: The `formatted` flag for `np2torch` will not be thread safe, calling `np2torch = Np2Torch()` per invocation should make it safe

In [7]:
# export
class Np2torch(Formatter):
    def predicate(self, arg): return isinstance(arg, np.ndarray)
    def formatter(self, arg): return torch.from_numpy(arg)
np2torch = Np2torch()

`numpyify` decorator will allow functions designed for torch tensors to also work for numpy tensors. 

NOTE: this will fail if input does not contain a `torch.tensor` (i.e. like the `eye` function, which takes an integer and returns a tensor). To ensure this works, one of the inputs must be a `torch.tensor`.

In [8]:
# export
def numpyify(f):
    def _numpyify(*args, **kwargs):
        np2torch = Np2torch()                      # For thread safety, make a local instantiation
        args, kwargs = np2torch((args, kwargs))
        out = f(*args, **kwargs)
        if np2torch.formatted: out = torch2np(out) # Convert back
        return out
    return _numpyify

# Tests

`assert_allclose` checks if two things, `A` and `B`, are close to each other. I use `np.allclose` because its more robust than `torch.allclose` (i.e. numpys version will work with datatypes other than `np.array`s, like `bool`s and `int`s)

NOTE: I'm assuming the format of the inputs is the same; if not I'm assuming this is programmer error.

In [9]:
# export
def _assert_allclose(A, B, **kwargs):
    if isinstance(A, tuple):            
        for a,b in zip(A,B): _assert_allclose(a, b, **kwargs) # Possibly add "strict" keyword here
    elif isinstance(A, dict):           
        for key in A.keys() | B.keys(): _assert_allclose(A[key], B[key], **kwargs)
    else:
        try:    assert(np.allclose(A, B, **kwargs))
        except: assert(np.all(A == B))

In [10]:
# export
def assert_allclose(A, B, **kwargs): _assert_allclose(*torch2np((A, B)), **kwargs)

In [11]:
A = torch.rand((4,3))
B = np.random.normal(size=(4,3))
A, B

(tensor([[0.3652, 0.2176, 0.0747],
         [0.2132, 0.6151, 0.7977],
         [0.3664, 0.1665, 0.7018],
         [0.9029, 0.1427, 0.5380]]),
 array([[ 0.48120975,  1.07263834, -0.56003356],
        [ 0.17232236, -0.55993703, -0.08343174],
        [-0.43181697, -0.57468503, -0.91940712],
        [-0.33962012,  2.19969209,  0.57362894]]))

In [12]:
assert_allclose(A, A+1e-5, atol=1e-5)
assert_allclose((A, (B, {'test': 1.})), (A+1e-5, (B+1e-5, {'test': 1 + 1e-5})), atol=1e-5)

Since we have multiple functions that should work for torch and numpy, we should have a way to test for both without having to write duplicate tests.

In [13]:
# export
def assert_allclose_f(f, x, y, **kwargs):
    if not isinstance(x, tuple): x = (x,)
    assert_allclose(f(*x), y, **kwargs)

In [14]:
# export 
def assert_allclose_f_ttn(f, x, y, **kwargs): # ttn == "torch, then numpy"
    torch2np = Torch2np()
    assert_allclose_f(f, x, y, **kwargs) # Torch test
    x, y = torch2np((x, y))
    assert(torch2np.formatted)           # Make sure something was converted
    assert_allclose_f(f, x, y, **kwargs) # Numpy test

# General

For some reason I can't find a built-in that will reverse and return a list without doing some iterable thing.

TODO: get this to work for torch tensors

In [15]:
# export
def reverse(A): return A[::-1]

In [16]:
assert_allclose(reverse(['a', 'b', 'c']), ['c', 'b', 'a'])

`shape` returns the tensor shape as a tensor the same type of the tensor. Convenient if you need to do arithmetic based on the size of the tensor.

In [17]:
# export
@numpyify
def shape(A, dtype=None):
    if dtype is None: dtype = A.dtype
    return A.new_tensor(A.shape, dtype=dtype)

In [18]:
A = torch.rand(3,4)
assert_allclose_f_ttn(shape, A, torch.FloatTensor([3, 4]))

In [19]:
# export
@numpyify
def stackify(A, dim=0):
    if isinstance(A, tuple): return torch.stack([stackify(a, dim) for a in A], dim)
    return A

In [20]:
x = (tuple(torch.FloatTensor([1,2])), tuple(torch.FloatTensor([1,2])))
assert_allclose_f_ttn(stackify, (x,), torch.FloatTensor([[1, 2],[1, 2]]))

pytorch doesnt have an equivalent of `np.delete`

NOTE: temporarily not `numpyified` because I need `np.array(,dtype=np.object)` to work since torch does not support jagged/nested tensors yet.

In [21]:
# export
def delete(A, idx_delete):
    idx = torch.ones(len(A), dtype=torch.bool)
    idx[idx_delete] = False
    return A[idx]

In [22]:
A = torch.FloatTensor([[1,2,3],
                       [4,5,6],
                       [7,8,9]])
assert_allclose_f_ttn(delete, (A, 1), torch.FloatTensor([[1, 2, 3],
                                                         [7, 8, 9]]))

In [23]:
# export
def rescale(A, r1, r2): return (A-r1[0])/(r1[1]-r1[0])*(r2[1]-r2[0])+r2[0]

In [24]:
A = torch.FloatTensor([1,2])
assert_allclose_f_ttn(rescale, (A, torch.FloatTensor([1,2]), torch.FloatTensor([2,4])), 
                      torch.FloatTensor([2, 4]))

# Points/Vectors

Kind of hard to keep separate distinction between points and vectors even though they technically aren't the same thing.

## General point stuff

`singlify` decorator will allow functions designed for multiple points to work for single point inputs. points should have a shape of `(N, [2,3])` whereas a single point will have shape of `([2,3])`, so its convenient to just define the function for multiple points and use the decorator so it will work for a single point.

NOTE: first argument must be `ps`

In [25]:
# export
def singlify(f):
    def _singlify(ps, *args, **kwargs):
        single = len(ps.shape) == 1
        if single: ps = ps[None]
        ps = f(ps, *args, **kwargs)
        if single: ps = ps[0]
        return ps
    return _singlify

`augment` will add ones to points; useful for affine and homography xforms

In [26]:
# export
@numpyify
@singlify
def augment(ps): return torch.cat([ps, ps.new_ones((len(ps), 1))], dim=1)

In [27]:
ps = torch.FloatTensor([[0.1940, 0.2536],
                        [0.2172, 0.1626],
                        [0.9834, 0.2700],
                        [0.5324, 0.7137]])
assert_allclose_f_ttn(augment, ps, torch.FloatTensor([[0.1940, 0.2536, 1.0000],
                                                      [0.2172, 0.1626, 1.0000],
                                                      [0.9834, 0.2700, 1.0000],
                                                      [0.5324, 0.7137, 1.0000]]))

`deaugment` will remove last column; might wanna add check to make sure column contains ones

In [28]:
# export
@singlify
def deaugment(ps): return ps[:, 0:-1]

In [29]:
ps = torch.FloatTensor([[0.1940, 0.2536, 1.0000],
                        [0.2172, 0.1626, 1.0000],
                        [0.9834, 0.2700, 1.0000],
                        [0.5324, 0.7137, 1.0000]])
assert_allclose_f_ttn(deaugment, ps, torch.FloatTensor([[0.1940, 0.2536],
                                                        [0.2172, 0.1626],
                                                        [0.9834, 0.2700],
                                                        [0.5324, 0.7137]]))

`normalize` will divide by last column and remove it

In [30]:
# export
@singlify
def normalize(ps): return deaugment(ps/ps[:, [-1]])

In [31]:
ps = torch.FloatTensor([[0.1940, 0.2536, 2.0000],
                        [0.2172, 0.1626, 3.0000],
                        [0.9834, 0.2700, 4.0000],
                        [0.5324, 0.7137, 5.0000]])
assert_allclose_f_ttn(normalize, ps, torch.FloatTensor([[0.0970, 0.1268],
                                                        [0.0724, 0.0542],
                                                        [0.2458, 0.0675],
                                                        [0.1065, 0.1427]]), atol=1e-4)

## Bounding box stuff

`ps_bb` is points bounding box

In [32]:
# export
@numpyify
def ps_bb(ps): return stackify((torch.min(ps, dim=0).values, torch.max(ps, dim=0).values))

In [33]:
ps = torch.FloatTensor([[0.1940, 0.2536],
                        [0.2172, 0.1626],
                        [0.9834, 0.2700],
                        [0.5324, 0.7137]])
assert_allclose_f_ttn(ps_bb, ps, torch.FloatTensor([[0.194 , 0.1626],
                                                    [0.9834, 0.7137]]))

`array_bb` is array bounding box

In [34]:
# export
@numpyify
def array_bb(arr, dtype=None):
    if dtype is None: dtype = arr.dtype
    return arr.new_tensor([[0,0], [arr.shape[1]-1, arr.shape[0]-1]], dtype=dtype)

In [35]:
arr = torch.rand(5,4)
assert_allclose_f_ttn(array_bb, arr, torch.FloatTensor([[0, 0],
                                                        [3, 4]]))

`bb_sz` returns the size of a bounding box

In [36]:
# export
@numpyify
def bb_sz(bb): return stackify((bb[1,1]-bb[0,1]+1, bb[1,0]-bb[0,0]+1))

In [37]:
bb = torch.FloatTensor([[0,0],[5,4]])
assert_allclose_f_ttn(bb_sz, bb, torch.FloatTensor([5, 6]))

`bb_grid` is bounding box grid; i,j is swapped to x,y

In [38]:
# export
@numpyify
def bb_grid(bb, dtype=None):
    if dtype is None: dtype = bb.dtype
    return stackify(reverse(torch.meshgrid(torch.arange(bb[0,1], bb[1,1]+1, dtype=dtype, device=bb.device), 
                                           torch.arange(bb[0,0], bb[1,0]+1, dtype=dtype, device=bb.device))))

In [39]:
bb = torch.FloatTensor([[0,0],[2,1]])
assert_allclose_f_ttn(bb_grid, bb, torch.FloatTensor([[[0, 1, 2],
                                                       [0, 1, 2]],
                                                      [[0, 0, 0],
                                                       [1, 1, 1]]]))

`bb_array` applies bounding box to array and returns the sub array

In [40]:
# export
@numpyify
def bb_array(arr, bb): 
    bb = bb.long() # Must be long for indexing; cannot round either incase input bb is type long
    return arr[bb[0,1]:bb[1,1]+1, bb[0,0]:bb[1,0]+1]

In [41]:
arr = torch.FloatTensor([[1,2,3],[4,5,6],[7,8,9]])
bb = torch.FloatTensor([[1,1],[2,2]])
assert_allclose_f_ttn(bb_array, (arr, bb), torch.FloatTensor([[5, 6],[8, 9]]))

In [42]:
# export
def is_p_in_bb(p, bb): return p[0] >= bb[0,0] and p[1] >= bb[0,1] and p[0] <= bb[1,0] and p[1] <= bb[1,1]

In [43]:
p1 = torch.FloatTensor([0.0, 0.0])
p2 = torch.FloatTensor([1.5, 1.5])
bb = torch.FloatTensor([[1,1], [2,2]])
assert_allclose_f_ttn(is_p_in_bb, (p1, bb), False)
assert_allclose_f_ttn(is_p_in_bb, (p2, bb), True)

In [44]:
# export
def is_bb_in_bb(bb1, bb2): return is_p_in_bb(bb1[0], bb2) and is_p_in_bb(bb1[1], bb2)

In [45]:
bb1 = torch.FloatTensor([[1,1],[5,5]])
bb2 = torch.FloatTensor([[2,2],[4,4]])
bb3 = torch.FloatTensor([[8,8],[9,9]])
assert_allclose_f_ttn(is_bb_in_bb, (bb2, bb1), True)
assert_allclose_f_ttn(is_bb_in_bb, (bb3, bb1), False)

## Boundary stuff

In [46]:
# export
def is_p_in_b(p, b): return Polygon(b).contains(Point(*p))

In [47]:
b  = torch.FloatTensor([[0,0],[0,1],[1,1],[1,0]])
p1 = torch.FloatTensor([0.5, 0.5])
p2 = torch.FloatTensor([1.5, 1.5])
assert_allclose_f_ttn(is_p_in_b, (p1, b), True)
assert_allclose_f_ttn(is_p_in_b, (p2, b), False)

In [48]:
# export
def bb2b(bb): return bb[[[0,0],[0,1],[1,1],[1,0]],
                        [[0,0],[0,1],[0,1],[0,0]]]

In [49]:
bb = torch.FloatTensor([[1,1],[5,5]])
assert_allclose_f_ttn(bb2b, bb, torch.FloatTensor([[1., 1.],
                                                   [1., 5.],
                                                   [5., 5.],
                                                   [5., 1.]]))

## Point geometries

`grid2ps` converts grid to points

In [50]:
# export
@numpyify
def grid2ps(X, Y, order='row'):
    if   order == 'row': return stackify((X.flatten(), Y.flatten()), dim=1)
    elif order == 'col': return grid2ps(X.T, Y.T, order='row')
    else: raise RuntimeError(f'Unrecognized option: {order}')

In [51]:
X = torch.FloatTensor([[1,2],[1,2]])
Y = torch.FloatTensor([[1,1],[2,2]])
assert_allclose_f_ttn(grid2ps, (X, Y, 'row'), torch.FloatTensor([[1, 1],
                                                                 [2, 1],
                                                                 [1, 2],
                                                                 [2, 2]]))
assert_allclose_f_ttn(grid2ps, (X, Y, 'col'), torch.FloatTensor([[1, 1],
                                                                 [1, 2],
                                                                 [2, 1],
                                                                 [2, 2]]))

In [52]:
# export
@numpyify
def array_ps(arr, dtype=None):
    if dtype is None: dtype = arr.dtype
    return grid2ps(*bb_grid(array_bb(arr), dtype))

In [53]:
arr = torch.rand(3,2)
assert_allclose_f_ttn(array_ps, arr, torch.FloatTensor([[0, 0],
                                                        [1, 0],
                                                        [0, 1],
                                                        [1, 1],
                                                        [0, 2],
                                                        [1, 2]]))

`crrgrid` is centered rectanglular rectangle grid

NOTE: not yet numpyified

In [54]:
# export
def crrgrid(num_h, num_w, spacing_h, spacing_w, dtype, device=None):
    h, w = spacing_h*(num_h-1), spacing_w*(num_w-1)
    y = torch.linspace(-h/2, h/2, int(num_h), dtype=dtype, device=device)
    x = torch.linspace(-w/2, w/2, int(num_w), dtype=dtype, device=device)
    return grid2ps(*reverse(torch.meshgrid(y, x)), 'col')

In [55]:
assert_allclose_f_ttn(crrgrid, (2, 2, 1, 1, torch.float), torch.FloatTensor([[-0.5000, -0.5000],
                                                                             [-0.5000,  0.5000],
                                                                             [ 0.5000, -0.5000],
                                                                             [ 0.5000,  0.5000]]))

`csrgrid` is centered square rectangle grid

NOTE: not yet numpyified

In [56]:
# export
def csrgrid(num_h, num_w, spacing, dtype, device=None):
    return crrgrid(num_h, num_w, spacing, spacing, dtype, device)

In [57]:
assert_allclose_f_ttn(csrgrid, (2, 2, 1, torch.float), torch.FloatTensor([[-0.5, -0.5],
                                                                          [-0.5,  0.5],
                                                                          [ 0.5, -0.5],
                                                                          [ 0.5,  0.5]]))

`csdgrid` is centered square diamond grid

NOTE: not yet numpyified

In [58]:
# export
def csdgrid(num_h, num_w, spacing, fo, dtype, device=None):
    h, w = spacing*(num_h-1), spacing*(num_w-1)
    xs_grid = torch.linspace(-w/2, w/2, int(num_w), dtype=dtype, device=device)
    ys_grid = torch.linspace(-h/2, h/2, int(num_h), dtype=dtype, device=device)
    ps = []
    for x_grid in xs_grid:
        if fo: ys, fo = ys_grid[0::2], False
        else:  ys, fo = ys_grid[1::2], True
        xs = torch.full((len(ys),), x_grid, dtype=dtype, device=device)
        ps.append(stackify((xs, ys), dim=1))
    return torch.cat(ps)

In [59]:
assert_allclose_f_ttn(csdgrid, (5, 4, 1, True, torch.float), torch.FloatTensor([[-1.5, -2],
                                                                                [-1.5,  0],
                                                                                [-1.5,  2],
                                                                                [-0.5, -1],
                                                                                [-0.5,  1],
                                                                                [ 0.5, -2],
                                                                                [ 0.5,  0],
                                                                                [ 0.5,  2],
                                                                                [ 1.5, -1],
                                                                                [ 1.5,  1]]))

`cfpgrid` is centered four point grid

NOTE: not yet numpyified

In [60]:
# export
def cfpgrid(h, w, dtype, device=None): return crrgrid(2, 2, h, w, dtype, device)

In [61]:
assert_allclose_f_ttn(cfpgrid, (2 ,2, torch.float), torch.FloatTensor([[-1, -1],
                                                                       [-1,  1],
                                                                       [ 1, -1],
                                                                       [ 1,  1]]))

## General vector stuff

`unitize` will make norm of each vector 1

NOTE: Handling of zero vector might be a little tricky

In [62]:
# export
@numpyify
@singlify
def unitize(vs): return vs/torch.norm(vs, dim=1, keepdim=True)

In [63]:
vs = torch.FloatTensor([[0.1940, 0.2536],
                        [0.2172, 0.1626],
                        [0.9834, 0.2700],
                        [0.5324, 0.7137]])
assert_allclose_f_ttn(unitize, vs, torch.FloatTensor([[0.6075896, 0.7942511],
                                                      [0.8005304, 0.5992921],
                                                      [0.9643143, 0.2647598],
                                                      [0.5979315, 0.8015471]]))

In [64]:
# export
@numpyify
def cross_mat(v):
    zero = v.new_tensor(0)
    
    return stackify((( zero, -v[2],  v[1]),
                     ( v[2],  zero, -v[0]),
                     (-v[1],  v[0],  zero)))

In [65]:
v = torch.FloatTensor([1,2,3])
assert_allclose_f_ttn(cross_mat, v, torch.FloatTensor([[ 0,-3, 2],
                                                       [ 3, 0,-1],
                                                       [-2, 1, 0]]))
assert_allclose(cross_mat(v)@v, 0)

## Transforms

`pmm` is point matrix multiplication

In [66]:
# export
@singlify
def pmm(ps, A, aug=False):
    if aug: ps = augment(ps)
    ps = (A@ps.T).T
    if aug: ps = normalize(ps) # works for both affine and homography transforms
    return ps

In [67]:
ps = torch.FloatTensor([[0.1940, 0.2536],
                        [0.2172, 0.1626],
                        [0.9834, 0.2700],
                        [0.5324, 0.7137]])
A = torch.FloatTensor([[0.9571, 0.5551],
                       [0.8914, 0.2626]])
assert_allclose_f_ttn(pmm, (ps, A), torch.FloatTensor([[0.3265, 0.2395],
                                                       [0.2981, 0.2363],
                                                       [1.0911, 0.9475],
                                                       [0.9057, 0.6620]]), atol=1e-4)

### Spherical coordinates

In [68]:
@numpyify
@singlify
def cart2spherical(ps):
    x, y, z = ps[:,0], ps[:,1], ps[:,2]
    r = torch.sqrt(x**2 + y**2 + z**2)
    theta = torch.atan2(torch.sqrt(x**2 + y**2), z)
    phi = torch.atan2(y, x)
    return stackify((r, theta, phi), 1)

In [69]:
ps = torch.FloatTensor([[0.4082785 , 0.1256695 , 0.54343185],
                        [0.93478803, 0.40557636, 0.40224384],
                        [0.48831708, 0.82743735, 0.68537884]])
assert_allclose_f_ttn(cart2spherical, ps, torch.FloatTensor([[0.69123247, 0.66619615, 0.29860039],
                                                             [1.09550032, 1.19482283, 0.40935945],
                                                             [1.18019079, 0.95116431, 1.03764654]]))

In [70]:
@numpyify
@singlify
def spherical2cart(ps):
    r, theta, phi = ps[:,0], ps[:,1], ps[:,2]
    x = r*torch.sin(theta)*torch.cos(phi)
    y = r*torch.sin(theta)*torch.sin(phi)
    z = r*torch.cos(theta)
    return stackify((x, y, z), dim=1)

In [71]:
ps = torch.FloatTensor([[0.69123247, 0.66619615, 0.29860039],
                        [1.09550032, 1.19482283, 0.40935945],
                        [1.18019079, 0.95116431, 1.03764654]])
assert_allclose_f_ttn(spherical2cart, ps, torch.FloatTensor([[0.4082785 , 0.1256695 , 0.5434318 ],
                                                             [0.9347881 , 0.4055764 , 0.40224388],
                                                             [0.4883171 , 0.82743734, 0.68537885]]))

In [72]:
ps = torch.rand(4,3)
ps

tensor([[0.1094, 0.0961, 0.4715],
        [0.8728, 0.4183, 0.0757],
        [0.7712, 0.6749, 0.8361],
        [0.6426, 0.1061, 0.6182]])

In [73]:
assert_allclose(ps, spherical2cart(cart2spherical(ps)))

### Point conditioning

`condition_mat` is typically used to "condition" points to improve conditioning; its inverse is usually applied afterwards. It sets the mean of the points to zero and the average distance to `sqrt(2)`. I use the term "condition" here so I don't get confused with "normalization" which is used above.

In [74]:
# export
@numpyify
def condition_mat(ps):
    zero, one = ps.new_tensor(0), ps.new_tensor(1)

    xs, ys = ps[:, 0], ps[:, 1]
    mean_x, mean_y = xs.mean(), ys.mean()
    s_m = math.sqrt(2)*len(ps)/(torch.sqrt((xs-mean_x)**2+(ys-mean_y)**2)).sum()
    return stackify((( s_m, zero, -mean_x*s_m),
                     (zero,  s_m, -mean_y*s_m),
                     (zero, zero,         one)))

In [75]:
ps = torch.FloatTensor([[0.5466, 0.0889],
                        [0.1493, 0.6591],
                        [0.5600, 0.0352],
                        [0.7287, 0.5892]])
assert_allclose_f_ttn(condition_mat, ps, torch.FloatTensor([[ 4.0950,  0.0000, -2.0317],
                                                            [ 0.0000,  4.0950, -1.4050],
                                                            [ 0.0000,  0.0000,  1.0000]]), atol=1e-4)

In [76]:
# export
def condition(ps):
    T = condition_mat(ps)
    return pmm(ps, T, aug=True), T

In [77]:
ps = torch.FloatTensor([[0.5466, 0.0889],
                        [0.1493, 0.6591],
                        [0.5600, 0.0352],
                        [0.7287, 0.5892]])
ps_cond, T = condition(ps)
assert_allclose(ps_cond.mean(), 0, atol=1e-6)
assert_allclose(ps_cond.norm(dim=1).mean(), math.sqrt(2), atol=1e-6)

### Homographies

`homography` estimates a homography between two sets of points

In [78]:
# export
@numpyify
def homography(ps1, ps2):    
    # Condition and augment points
    (ps1_cond, T1), (ps2_cond, T2) = map(condition, [ps1, ps2])
    ps1_cond, ps2_cond = map(augment, [ps1_cond, ps2_cond])

    # Form homogeneous system
    L = torch.cat([torch.cat([ps1_cond, torch.zeros_like(ps1_cond), -ps2_cond[:, 0:1]*ps1_cond], dim=1),
                   torch.cat([torch.zeros_like(ps1_cond), ps1_cond, -ps2_cond[:, 1:2]*ps1_cond], dim=1)])

    # Solution is the last column of V
    H12_cond = torch.svd(L, some=False).V[:,-1].reshape(3,3)

    # Undo conditioning
    H12 = torch.inverse(T2)@H12_cond@T1
    H12 = H12/H12[2,2] # Sets H12[2,2] to 1
    return H12

In [79]:
ps1 = torch.FloatTensor([[-350, -350],
                         [-350,  350],
                         [ 350, -350],
                         [ 350,  350]])
ps2 = torch.FloatTensor([[ 970,  517],
                         [ 156,  498],
                         [ 973, 1317],
                         [ 192, 1279]])
assert_allclose_f_ttn(homography, (ps1, ps2), 
                      torch.FloatTensor([[ 6.252e-02, -1.117e+00,  5.6786e+02],
                                         [ 1.183e+00, -8.049e-03,  9.1087e+02],
                                         [ 6.000e-05,  3.650e-05,  1.0000e+00]]), atol=1e-3)

### Rotations

`approx_R` gives the nearest rotational approximation to the input matrix (frobenius norm?). Note that for a proper rotation determinant must be +1, which is checked after.

In [80]:
# export
@numpyify
def approx_R(R):
    one = R.new_tensor(1)
    
    [U,_,V] = torch.svd(R)
    R = U@V.T
    if not torch.isclose(torch.det(R), one):
        R = R.new_full((3,3), math.nan)
    return R

In [81]:
R = torch.FloatTensor([[0.0958, 0.8441, 0.2009],
                       [0.7877, 0.9110, 0.9277],
                       [0.3727, 0.7262, 0.1417]])
assert_allclose_f_ttn(approx_R, R, torch.FloatTensor([[-0.5659,  0.8152,  0.1237],
                                                      [ 0.5155,  0.2328,  0.8247],
                                                      [ 0.6434,  0.5304, -0.5520]]), atol=1e-4)

#### Euler

In [82]:
# export
@numpyify
def euler2R(euler):
    s, c = torch.sin, torch.cos

    e_x, e_y, e_z = euler
    return stackify((
        (c(e_y)*c(e_z), c(e_z)*s(e_x)*s(e_y) - c(e_x)*s(e_z), s(e_x)*s(e_z) + c(e_x)*c(e_z)*s(e_y)),
        (c(e_y)*s(e_z), c(e_x)*c(e_z) + s(e_x)*s(e_y)*s(e_z), c(e_x)*s(e_y)*s(e_z) - c(e_z)*s(e_x)),
        (      -s(e_y),                        c(e_y)*s(e_x),                        c(e_x)*c(e_y))
    ))

In [83]:
euler = torch.FloatTensor([0.2748, 0.0352, 0.4496])
assert_allclose_f_ttn(euler2R, euler, torch.FloatTensor([[ 0.9001, -0.4097,  0.1484],
                                                         [ 0.4343,  0.8710, -0.2297],
                                                         [-0.0352,  0.2712,  0.9619]]), atol=1e-4)

In [84]:
# export
@numpyify
def R2euler(R):
    return stackify((torch.atan2( R[2, 1], R[2, 2]),
                     torch.atan2(-R[2, 0], torch.sqrt(R[0, 0]**2+R[1, 0]**2)),
                     torch.atan2( R[1, 0], R[0, 0])))

In [85]:
R = torch.FloatTensor([[ 0.9001, -0.4097,  0.1484],
                       [ 0.4343,  0.8710, -0.2297],
                       [-0.0352,  0.2712,  0.9619]])
assert_allclose_f_ttn(R2euler, R, torch.FloatTensor([0.2748, 0.0352, 0.4496]), atol=1e-4) 

ensure euler => R => euler

In [86]:
euler = torch.FloatTensor([0.2748, 0.0352, 0.4496])
assert_allclose(R2euler(euler2R(euler)), euler)

#### Rodrigues

rodrigues conversion from:
* https://www2.cs.duke.edu/courses/fall13/compsci527/notes/rodrigues.pdf

In [87]:
# export
@numpyify
def rodrigues2R(r):
    zero = r.new_tensor(0)
    
    theta = torch.norm(r)
    if theta > math.pi: warnings.warn('Theta greater than pi')
    if torch.isclose(theta, zero): return torch.eye(3, dtype=r.dtype, device=r.device)
    u = r/theta
    return torch.eye(3, dtype=r.dtype, device=r.device)*torch.cos(theta) + \
           (1-torch.cos(theta))*u[:,None]@u[:,None].T + \
           cross_mat(u)*torch.sin(theta)

In [88]:
theta = math.pi/4
k = torch.FloatTensor([ 0.8155,  0.0937, -0.5711])
r = theta*k
assert_allclose_f_ttn(rodrigues2R, r, torch.FloatTensor([[ 0.9019,  0.4262, -0.0702],
                                                         [-0.3814,  0.7097, -0.5923],
                                                         [-0.2027,  0.5610,  0.8026]]), atol=1e-4)

In [89]:
# export
@numpyify
def R2rodrigues(R):
    zero, one, pi = R.new_tensor(0), R.new_tensor(1), R.new_tensor(math.pi)
    
    A = (R-R.T)/2
    rho = A[[2,0,1],[1,2,0]]
    s = torch.norm(rho)
    c = (R.trace()-1)/2
    if torch.isclose(s, zero) and torch.isclose(c, one): 
        r = R.new_zeros(3)
    elif torch.isclose(s, zero) and torch.isclose(c, -one):
        V = R + torch.eye(3, dtype=R.dtype, device=R.device)
        v = V[:, torch.where(~torch.isclose(torch.norm(V, dim=0), zero))[0][0]] # Just get first non-zero
        u = unitize(v)
        def S_half(r):
            if torch.isclose(torch.norm(r), pi) and \
               ((torch.isclose(r[0], zero) and torch.isclose(r[1], zero) and r[2] < 0) or \
                (torch.isclose(r[0], zero) and r[1] < 0) or \
                (r[0] < 0)):
                return -r
            else: 
                return r
        r = S_half(u*math.pi)
    elif not torch.isclose(s, zero):
        u = rho/s
        theta = torch.atan2(s,c)
        r = u*theta
    else: raise RuntimeError('This shouldnt happen; please debug')
    return r

In [90]:
R = torch.FloatTensor([[ 0.9019,  0.4262, -0.0702],
                       [-0.3814,  0.7097, -0.5923],
                       [-0.2027,  0.5610,  0.8026]])
assert_allclose_f_ttn(R2rodrigues, R, torch.FloatTensor([ 0.6405,  0.0736, -0.4485]), atol=1e-4)

In [91]:
r = torch.FloatTensor([.1,.2,.3])
assert_allclose(R2rodrigues(rodrigues2R(r)), r)

#### Other rotation stuff

`approx_R` gives the nearest rotational approximation to the input matrix. Note that for a proper rotation determinant must be +1, which is checked after.

In [92]:
# export
@numpyify
def approx_R(R):
    one = R.new_tensor(1)
    
    [U,_,V] = torch.svd(R)
    R = U@V.T
    if not torch.isclose(torch.det(R), one):
        R = R.new_full((3,3), math.nan)
    return R

In [93]:
R = torch.FloatTensor([[0.0958, 0.8441, 0.2009],
                       [0.7877, 0.9110, 0.9277],
                       [0.3727, 0.7262, 0.1417]])
assert_allclose_f_ttn(approx_R, R, torch.FloatTensor([[-0.5659,  0.8152,  0.1237],
                                                      [ 0.5155,  0.2328,  0.8247],
                                                      [ 0.6434,  0.5304, -0.5520]]), atol=1e-4)

### Rigid

Rigid transforms are a rotation followed by translation.

In [94]:
# export
@numpyify
def Rt2M(R, t):
    M = torch.cat([R, t[:,None]], dim=1)
    M = torch.cat([M, M.new_tensor([[0,0,0,1]])])
    return M

In [95]:
R = torch.FloatTensor([[-0.5659,  0.8152,  0.1237],
                       [ 0.5155,  0.2328,  0.8247],
                       [ 0.6434,  0.5304, -0.5520]])
t = torch.FloatTensor([1,2,3])
assert_allclose_f_ttn(Rt2M, (R,t), torch.FloatTensor([[-0.5659,  0.8152,  0.1237,  1.0000],
                                                      [ 0.5155,  0.2328,  0.8247,  2.0000],
                                                      [ 0.6434,  0.5304, -0.5520,  3.0000],
                                                      [ 0.0000,  0.0000,  0.0000,  1.0000]]))

In [96]:
# export
def M2Rt(M): return M[0:3,0:3], M[0:3,3]

In [97]:
M = torch.FloatTensor([[-0.5659,  0.8152,  0.1237,  1.0000],
                       [ 0.5155,  0.2328,  0.8247,  2.0000],
                       [ 0.6434,  0.5304, -0.5520,  3.0000],
                       [ 0.0000,  0.0000,  0.0000,  1.0000]])
assert_allclose_f_ttn(M2Rt, M, (torch.FloatTensor([[-0.5659,  0.8152,  0.1237],
                                                   [ 0.5155,  0.2328,  0.8247],
                                                   [ 0.6434,  0.5304, -0.5520]]), torch.FloatTensor([1,2,3])))

In [98]:
# export
def invert_rigid(M):
    R, t = M2Rt(M)
    return Rt2M(R.T, -R.T@t)

In [99]:
M = torch.FloatTensor([[-0.5659,  0.8152,  0.1237,  1.0000],
                       [ 0.5155,  0.2328,  0.8247,  2.0000],
                       [ 0.6434,  0.5304, -0.5520,  3.0000],
                       [ 0.0000,  0.0000,  0.0000,  1.0000]])
assert_allclose_f_ttn(invert_rigid, M, torch.inverse(M), atol=1e-3)

In [100]:
# export
def mult_rigid(M1, M2):
    R1, t1 = M2Rt(M1)
    R2, t2 = M2Rt(M2)
    return Rt2M(R1@R2, R1@t2+t1)

In [101]:
M = torch.FloatTensor([[-0.5659,  0.8152,  0.1237,  1.0000],
                       [ 0.5155,  0.2328,  0.8247,  2.0000],
                       [ 0.6434,  0.5304, -0.5520,  3.0000],
                       [ 0.0000,  0.0000,  0.0000,  1.0000]])
assert_allclose_f_ttn(mult_rigid, (M,M), M@M)

## More Vector stuff

Stuff placed here mainly because it requires stuff from transforms section.

For a random unit vector uniformly sampled over a sphere, if you randomly sample `theta` and `phi` directly, you will over sample near the poles. Instead, you can do an "equal area" projection of the sphere onto a cylinder (i.e. rectangle), uniformly sample the cylinder/rectangle, and then project the point back. More info here:
* https://math.stackexchange.com/a/44691

NOTE: `torch.rand` is random uniform between `[0,1)`, so it is rescaled as needed. Also, since `random_unit` doesnt take in a `tensor`, it cannot be `numpyified`

In [102]:
# export
def random_unit(dtype=None, device=None):
    r = torch.ones(1, dtype=dtype, device=device)[0]
    theta = torch.acos(rescale(torch.rand(1, dtype=dtype, device=device)[0], [0,1], [-1,1]))
    phi = rescale(torch.rand(1, dtype=dtype, device=device)[0], [0,1], [0, 2*math.pi])
    return spherical2cart(stackify((r, theta, phi)))

In [103]:
assert_allclose(torch.norm(random_unit()), 1)

`v_v_angle` is vector-vector angle and returns the angle between two vectors

In [104]:
# export
@numpyify
def v_v_angle(a, b):   
    x = torch.dot(a,b)/(torch.norm(a)*torch.norm(b))
    return torch.acos(x.clamp(-1, 1)) # Precision errors can make this go outside domain [-1,1] => NaNs

In [105]:
a = torch.FloatTensor([ 0.5568, -0.4851, -0.6743])
b = torch.FloatTensor([-0.8482, -0.4175, -0.3260])
assert_allclose_f_ttn(v_v_angle, (a, b), 1.6207424465344742, atol=1e-5)
assert_allclose_f_ttn(v_v_angle, (a, a), 0, atol=1e-5)

`v_v_R` is vector vector rotation matrix and returns the rotation matrix between the two vectors

In [106]:
# export
@numpyify
def v_v_R(v1, v2):
    zero = v1.new_tensor(0)

    theta = v_v_angle(v1, v2)
    if torch.isclose(theta, zero): return torch.eye(3, dtype=v1.dtype, device=v1.device)
    v3 = unitize(torch.cross(v1, v2))
    return rodrigues2R(theta*v3)

In [107]:
v1 = torch.FloatTensor([ 0.5568, -0.4851, -0.6743])
v2 = torch.FloatTensor([-0.8482, -0.4175, -0.3260])
assert_allclose_f_ttn(v_v_R, (v1,v2), torch.FloatTensor([[-0.03390428,  0.54606884,  0.83705395],
                                                         [-0.74174788,  0.54757324, -0.3872643 ],
                                                         [-0.66982131, -0.63401291,  0.38648033]]))
assert_allclose_f_ttn(v_v_R, (v1,v1), torch.FloatTensor([[1, 0, 0],
                                                         [0, 1, 0],
                                                         [0, 0, 1]]))
assert_allclose(unitize(pmm(v1, v_v_R(v1,v2))), unitize(v2), atol=1e-4) 

# Line stuff

In [108]:
# export
@numpyify
def pm2l(p, m):
    zero, one = p.new_tensor(0), p.new_tensor(1)
    
    x, y = p
    if not torch.isfinite(m): a, b, c = one, zero,    -x
    else:                     a, b, c =   m, -one, y-m*x
    return stackify((a,b,c))

In [109]:
p = torch.FloatTensor([1.5, 2.5])
m = torch.FloatTensor([0.5])[0]
assert_allclose_f_ttn(pm2l, (p, m), torch.FloatTensor([ 0.5000, -1.0000,  1.7500]))

In [110]:
# export
@numpyify
def ps2l(p1, p2):
    zero = p1.new_tensor(0)

    x1, y1 = p1
    x2, y2 = p2
    if not torch.isclose(x2-x1, zero): m = (y2-y1)/(x2-x1)
    else:                              m = p1.new_tensor(math.inf)
    return pm2l(p1, m)

In [111]:
p1 = torch.FloatTensor([1.5, 2.5])
p2 = torch.FloatTensor([2.5, 3.5])
assert_allclose_f_ttn(ps2l, (p1, p2), torch.FloatTensor([ 1.0000,  -1.0000, 1.000]))

In [112]:
# export
@numpyify
def pld(p, l):
    x, y = p
    a, b, c = l
    return torch.abs(a*x + b*y + c)/torch.sqrt(a**2 + b**2)

In [113]:
p = torch.FloatTensor([0.5, 1.5])
l = torch.FloatTensor([1.0000,  0.0000, -1.5000])
assert_allclose_f_ttn(pld, (p, l), 1)

In [114]:
# export
@numpyify
def l_l_intersect(l1, l2):
    a1, b1, c1 = l1
    a2, b2, c2 = l2
    return stackify(((-c1*b2 + b1*c2)/(a1*b2 - b1*a2), (-a1*c2 + c1*a2)/(a1*b2 - b1*a2)))

In [115]:
l1 = torch.FloatTensor([.1, .2, .3])
l2 = torch.FloatTensor([.4, .2, .2])
assert_allclose_f_ttn(l_l_intersect, (l1, l2), torch.FloatTensor([ 0.3333, -1.6667]), atol=1e-4)

`b_ls` gets lines of boundary

In [116]:
# export
@numpyify
def b_ls(b): 
    return stackify(tuple(ps2l(b[idx], b[torch.remainder(idx+1, len(b))]) for idx in torch.arange(len(b))))

In [117]:
b = torch.FloatTensor([[1., 1.],
                       [1., 4.],
                       [5., 4.],
                       [5., 1.]])
assert_allclose_f_ttn(b_ls, b, torch.FloatTensor([[ 1.,  0., -1.],
                                                  [ 0., -1.,  4.],
                                                  [ 1.,  0., -5.],
                                                  [-0., -1.,  1.]]))

`bb_ls` gets the lines of an input bounding box

In [118]:
# export
@numpyify
def bb_ls(bb): return stackify((ps2l(bb[[0,0],[0,1]], bb[[0,1],[0,1]]),
                                ps2l(bb[[0,0],[0,1]], bb[[1,0],[0,1]]),
                                ps2l(bb[[0,1],[0,1]], bb[[1,1],[0,1]]),
                                ps2l(bb[[1,0],[0,1]], bb[[1,1],[0,1]])))

In [119]:
bb = torch.FloatTensor([[1, 2], [5, 4]])
assert_allclose_f_ttn(bb_ls, bb, torch.FloatTensor([[ 1,  0, -1],
                                                    [ 0, -1,  2],
                                                    [ 0, -1,  4],
                                                    [ 1,  0, -5]]))

TODO: handle edge cases

In [120]:
# export
@numpyify
def bb_l_intersect(bb, l):
    ls_bb = bb_ls(bb)
    ps = []
    for l_bb in ls_bb:
        p = l_l_intersect(l_bb, l)
        if (torch.isclose(p[0], bb[0,0]) or p[0] > bb[0,0]) and \
           (torch.isclose(p[0], bb[1,0]) or p[0] < bb[1,0]) and \
           (torch.isclose(p[1], bb[0,1]) or p[1] > bb[0,1]) and \
           (torch.isclose(p[1], bb[1,1]) or p[1] < bb[1,1]):
            ps.append(p)
    return stackify(tuple(ps))

In [121]:
bb = torch.FloatTensor([[-100,-100],[200,200]])
l = torch.FloatTensor([.1, .2, .3])
assert_allclose_f_ttn(bb_l_intersect, (bb, l), torch.FloatTensor([[-100.0000,   48.5000],
                                                                  [ 197.0000, -100.0000]]))

# Ellipse stuff

`sample_2pi` prevents accidentally resampling 2pi twice by linspacing with an additional sample and then removing the last sample

NOTE: Get numpy version working

In [122]:
# export
def sample_2pi(num_samples, dtype=None, device=None): 
    return torch.linspace(0, 2*math.pi, int(num_samples)+1, dtype=dtype, device=device)[:-1]

In [123]:
assert_allclose_f_ttn(sample_2pi, 3, torch.FloatTensor([0.0000, 2.0944, 4.1888]), atol=1e-4)

In [124]:
# export
@numpyify
def sample_ellipse(e, num_samples):
    h, k, a, b, alpha = e
    thetas = sample_2pi(num_samples, e.dtype, e.device)
    return stackify((a*torch.cos(alpha)*torch.cos(thetas) - b*torch.sin(alpha)*torch.sin(thetas) + h,
                     a*torch.sin(alpha)*torch.cos(thetas) + b*torch.cos(alpha)*torch.sin(thetas) + k), dim=1)

In [125]:
e = torch.FloatTensor([1, 2, 3, 4, math.pi/4])
assert_allclose_f_ttn(sample_ellipse, (e, 3), torch.FloatTensor([[ 3.1213,  4.1213],
                                                                 [-2.5101,  3.3888],
                                                                 [ 2.3888, -1.5101]]), atol=1e-4)

In [126]:
# export
@numpyify
def ellipse2conic(e):
    h, k, a, b, alpha = e
    A = a**2*torch.sin(alpha)**2 + b**2*torch.cos(alpha)**2
    B = 2*(b**2 - a**2)*torch.sin(alpha)*torch.cos(alpha)
    C = a**2*torch.cos(alpha)**2 + b**2*torch.sin(alpha)**2
    D = -2*A*h - B*k
    E = -B*h - 2*C*k
    F = A*h**2 + B*h*k + C*k**2 - a**2*b**2
    return stackify(((  A, B/2, D/2),
                     (B/2,   C, E/2),
                     (D/2, E/2,   F)))

In [127]:
e = torch.FloatTensor([1, 2, 3, 4, math.pi/4])
assert_allclose_f_ttn(ellipse2conic, e, torch.FloatTensor([[ 12.5000,   3.5000, -19.5000],
                                                           [  3.5000,  12.5000, -28.5000],
                                                           [-19.5000, -28.5000, -67.5000]]))

In [128]:
# export
@numpyify
def conic2ellipse(Aq):
    zero, pi = Aq.new_tensor(0), Aq.new_tensor(math.pi)
    
    A = Aq[0, 0]
    B = 2*Aq[0, 1]
    C = Aq[1, 1]
    D = 2*Aq[0, 2]
    E = 2*Aq[1, 2]
    F = Aq[2, 2]

    # Return nans if input conic is not ellipse
    if torch.any(~torch.isfinite(Aq)) or torch.isclose(B**2-4*A*C, zero) or B**2-4*A*C > 0:
        return Aq.new_full((5,), math.nan)

    # Equations below are from https://math.stackexchange.com/a/820896/39581

    # "coefficient of normalizing factor"
    q = 64*(F*(4*A*C-B**2)-A*E**2+B*D*E-C*D**2)/(4*A*C-B**2)**2

    # distance between center and focal point
    s = 1/4*torch.sqrt(torch.abs(q)*torch.sqrt(B**2+(A-C)**2))

    # ellipse parameters
    h = (B*E-2*C*D)/(4*A*C-B**2)
    k = (B*D-2*A*E)/(4*A*C-B**2)
    a = 1/8*torch.sqrt(2*torch.abs(q)*torch.sqrt(B**2+(A-C)**2)-2*q*(A+C))
    b = torch.sqrt(a**2-s**2)
    # Get alpha; note that range of alpha is [0, pi)
    if torch.isclose(q*A-q*C, zero) and torch.isclose(q*B, zero): alpha = zero # Circle
    elif torch.isclose(q*A-q*C, zero) and q*B > 0:                alpha = 1/4*pi
    elif torch.isclose(q*A-q*C, zero) and q*B < 0:                alpha = 3/4*pi
    elif q*A-q*C > 0 and (torch.isclose(q*B, zero) or q*B > 0):   alpha = 1/2*torch.atan(B/(A-C))
    elif q*A-q*C > 0 and q*B < 0:                                 alpha = 1/2*torch.atan(B/(A-C)) + pi
    elif q*A-q*C < 0:                                             alpha = 1/2*torch.atan(B/(A-C)) + 1/2*pi
    else: raise RuntimeError('"Impossible" condition reached; please debug')

    return stackify((h, k, a, b, alpha))

In [129]:
Aq = torch.FloatTensor([[  9.56324965,  -1.90407389,  -5.75510187],
                        [ -1.90407389,  15.43675035, -28.96942682],
                        [ -5.75510187, -28.96942682, -80.3060445 ]])
assert_allclose_f_ttn(conic2ellipse, Aq, 
                      torch.FloatTensor([1.0000, 2.0000, 4.0000, 3.0000, 0.2876]), atol=1e-4)
assert_allclose(ellipse2conic(conic2ellipse(Aq)), Aq)

# General image processing

In [130]:
# export
def rgb2gray(arr): # From Pillow documentation
    return arr[:,:,0]*(299/1000) + arr[:,:,1]*(587/1000) + arr[:,:,2]*(114/1000)

In [131]:
arr = torch.FloatTensor([[[.1,.2,.3],[.2,.2,.2]]])
assert_allclose_f_ttn(rgb2gray, arr, torch.FloatTensor([[0.1815, 0.2000]]))

In [132]:
# export
@numpyify
def imresize(arr, sz, mode='bilinear', align_corners=True):
    if not isinstance(sz, tuple): sz = tuple((shape(arr)//(shape(arr)/sz).min()).long())
    return torch.nn.functional.interpolate(arr[None, None, :, :], 
                                           size=sz, 
                                           mode=mode, 
                                           align_corners=align_corners).squeeze(0).squeeze(0)

In [133]:
arr = torch.FloatTensor([[1,1,1,1],
                         [2,2,2,2]])
assert_allclose_f_ttn(imresize, (arr, 3), 
                      torch.FloatTensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
                                         [1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000],
                                         [2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000]]))

`conv2d` is actually cross correlation, but you can transpose and do the same operation. This is mainly just a helper function to do 2d convolutions easily.

In [134]:
# export
@numpyify
def conv2d(arr, kernel, **kwargs):
    return torch.nn.functional.conv2d(arr[None,None], kernel[None, None], **kwargs).squeeze(0).squeeze(0)

In [135]:
arr = torch.FloatTensor([[1,2,3],
                         [3,2,1],
                         [1,1,1]])
kernel = torch.FloatTensor([[-0.25, 0.25],
                            [-0.25, 0.25]])
assert_allclose_f_ttn(conv2d, (arr, kernel), torch.FloatTensor([[ 0.0000,  0.0000],
                                                                [-0.2500, -0.2500]]))

`pad` is just a helper function to do 2d convolutions easily

In [136]:
# export
@numpyify
def pad(arr, pad, mode):
    return torch.nn.functional.pad(arr[None,None], pad, mode=mode).squeeze(0).squeeze(0)

In [137]:
arr = torch.FloatTensor([[1,2,3],
                         [3,2,1],
                         [1,1,1]])
assert_allclose_f_ttn(pad, (arr, (1,1,1,1), 'replicate'), torch.FloatTensor([[1,1,2,3,3],
                                                                             [1,1,2,3,3],
                                                                             [3,3,2,1,1],
                                                                             [1,1,1,1,1],
                                                                             [1,1,1,1,1]]))

`grad_array` by default uses replication to help with gradients along the edges. The default padding in `nn.functional.conv2d` is zero padding, which results in bad gradients along the edges.

In [138]:
# export
@numpyify
def grad_array(arr):
    kernel_sobel = arr.new_tensor([[-0.1250, 0, 0.1250],
                                   [-0.2500, 0, 0.2500],
                                   [-0.1250, 0, 0.1250]])
    arr = pad(arr, pad=(1,1,1,1), mode='replicate')
    return tuple(conv2d(arr, kernel) for kernel in (kernel_sobel, kernel_sobel.T))

In [139]:
arr = torch.FloatTensor([[1,2,3],
                         [3,2,1],
                         [1,1,1]])
assert_allclose_f_ttn(grad_array, arr, (torch.FloatTensor([[ 0.2500,  0.5000,  0.2500],
                                                           [-0.1250, -0.2500, -0.1250],
                                                           [-0.1250, -0.2500, -0.1250]]), 
                                        torch.FloatTensor([[ 0.7500,  0.0000, -0.7500],
                                                           [-0.1250, -0.5000, -0.8750],
                                                           [-0.8750, -0.5000, -0.1250]])))

In [140]:
# export
@numpyify
def interp_array(arr, ps, align_corners=True, **kwargs):    
    ps = stackify((rescale(ps[:, 0], [0, arr.shape[1]-1], [-1, 1]),
                   rescale(ps[:, 1], [0, arr.shape[0]-1], [-1, 1])), dim=1) # ps must be rescaled to [-1,1]
    return torch.nn.functional.grid_sample(arr[None, None], 
                                           ps[None, None], 
                                           align_corners=align_corners,
                                           **kwargs).squeeze()

In [141]:
arr = torch.FloatTensor([[1,2,3],
                         [4,5,6],
                         [7,8,9]])
ps = array_ps(arr)*0.8
assert_allclose_f_ttn(interp_array, (arr, ps), torch.FloatTensor([1.,1.8,2.6,3.4,4.2,5.,5.8,6.6,7.4]))

# Optimization stuff

`wlstsq` is weighted least squares

In [142]:
# export
@numpyify
def wlstsq(A, b, W=None):
    single = len(b.shape) == 1
    if single: b = b[:, None]
    if W is not None: # Weight matrix is a diagonal matrix with sqrt of the input weights
        W = torch.sqrt(W.reshape(-1,1))
        A, b = A*W, b*W
    x = torch.lstsq(b, A).solution[:A.shape[1],:] # first n rows contains solution
    if single: x = x.squeeze(1)
    return x

In [143]:
A = torch.FloatTensor([[1, 2, 3],
                       [2, 3, 4],
                       [4, 2, 5],
                       [3, 3, 2],
                       [1, 6, 7]])
b = torch.FloatTensor([[1, 2],
                       [1, 2],
                       [1, 2],
                       [2, 3],
                       [7, 3]])
W = torch.FloatTensor([1, 2, 3, 4, 5])
assert_allclose_f_ttn(wlstsq, (A, b, W), torch.FloatTensor([[-0.5300,  0.4480],
                                                            [ 1.0744,  0.6981],
                                                            [ 0.1175, -0.2283]]), atol=1e-4)

# Plotting

In [144]:
# export
def get_colors(n): return sns.color_palette(None, n)

# Notebook stuff

These are kind of hacky, but I like being able to rerun a notebook and have it auto save/build/convert at the end

In [145]:
# export
def get_notebook_file():
    id_kernel = re.search('kernel-(.*).json', ipykernel.connect.get_connection_file()).group(1)
    for server in list_running_servers():
        response = requests.get(requests.compat.urljoin(server['url'], 'api/sessions'),
                                params={'token': server.get('token', '')})
        for r in json.loads(response.text):
            if 'kernel' in r and r['kernel']['id'] == id_kernel:
                return Path(r['notebook']['path'])

In [146]:
assert_allclose(get_notebook_file().as_posix(), 'utils.ipynb')

In [147]:
# export
def save_notebook():
    file_notebook = get_notebook_file()
    _get_md5 = lambda : hashlib.md5(file_notebook.read_bytes()).hexdigest() 
    md5_start = _get_md5()
    display(Javascript('IPython.notebook.save_checkpoint();')) # Asynchronous
    while md5_start == _get_md5(): time.sleep(1e-1)

In [148]:
# export
def build_notebook(save=True):
    if save: save_notebook()
    nbdev.export.notebook2script(fname=get_notebook_file().as_posix())

In [149]:
# export
def convert_notebook(save=True, t='markdown'):
    if save: save_notebook()
    os.system(f'jupyter nbconvert --to {t} {get_notebook_file().as_posix()}')

# Build

In [150]:
build_notebook()

<IPython.core.display.Javascript object>

Converted utils.ipynb.
